In [1]:
%matplotlib inline
from abmdb_est import *

In [2]:
s_dict = {'room air conditioners':(np.arange(1949, 1962), 
                [96, 195, 238, 380, 1045, 1230, 1267, 1828, 1586, 1673, 1800, 1580, 1500]),
        'color televisions':(np.arange(1963,1971),
                [747,1480,2646,5118,5777,5982,5962,4631]),
        'clothers dryers':(np.arange(1949,1962),
                [106,319,492,635,737,890,1397,1523,1294,1240,1425,1260,1236])}

In [3]:
client = MongoClient()
database = client.abmDiffusion
prj = database.abmDatabase

In [5]:
id_cont = []
for x in prj.find({}, projection={"_id":1}):
    id_cont.append(x["_id"])

- 预测

In [20]:
s = s_dict["room air conditioners"][1]
data_cont = []
for i, _id in enumerate(id_cont):
    db = prj.find_one({"_id": _id})
    abm = abmDBfit(db)
    pred = abm.predict(s)
    mean_pred = np.mean(pred, axis=0)
    data_cont.append(list(mean_pred))

In [8]:
columns = ["mpa", "mad", "mse"]

In [21]:
df = pd.DataFrame(data_cont, index=id_cont, columns=columns)

In [22]:
df

,mpa,mad,mse
"gnm_random_graph(10000,80000)",0.278532,426.230767,256104.040461
"exponential_graph(10000,3)",0.323632,498.753170,387655.031996
"watts_strogatz_graph(10000,6,1.0)",0.245686,371.157982,185395.190230
"gnm_random_graph(10000,30000)",0.240320,368.527344,167256.574819
"watts_strogatz_graph(10000,6,0.3)",0.235251,357.425713,161246.793921
"gnm_random_graph(10000,40000)",0.238661,363.870929,177130.183431
"gnm_random_graph(10000,100000)",0.220013,352.658074,144849.102651
"gnm_random_graph(10000,50000)",0.248631,379.170061,203784.080193
"watts_strogatz_graph(10000,6,0.5)",0.350075,538.395183,423950.040436
"watts_strogatz_graph(10000,6,0)",0.156821,259.027758,84108.661003


In [23]:
df.to_excel(excel_writer="predict_room_air_conditioners.xls")

- 综合预测combine

In [6]:
abm_list = []
for _id in id_cont:
    db = prj.find_one({"_id": _id})
    abm_list.append(abmDBfit(db))

In [21]:
def combine_predict(s, b_idx=6, abm_list=abm_list):
    mean_one_step = []
    for i in range(b_idx, len(s)):
        x = s[:i]
        one_step = []
        for abm in abm_list:
            res = abm.fit(x)
            one_step.append(res[1][i])
            
        delta = abs(np.mean(one_step) - s[i])
        mean_one_step.append([delta/s[i], delta, delta**2])
    return mean_one_step

In [30]:
s = s_dict["room air conditioners"][1]
result_1 = combine_predict(s)
r_1 = np.mean(result_1, axis=0)
print(f"mpe:{r_1[0]:.3f}, mad:{r_1[1]:.0f}, mse:{r_1[2]:.2E}")

mpe:0.222, mad:341, mse:1.50E+05


In [31]:
s = s_dict["color televisions"][1]
result_2 = combine_predict(s)
r_2 = np.mean(result_2, axis=0)
print(f"mpe:{r_2[0]:.3f}, mad:{r_2[1]:.0f}, mse:{r_2[2]:.2E}")

mpe:0.045, mad:218, mse:6.21E+04


In [32]:
s = s_dict["clothers dryers"][1]
result_3 = combine_predict(s)
r_3 = np.mean(result_3, axis=0)
print(f"mpe:{r_3[0]:.3f}, mad:{r_3[1]:.0f}, mse:{r_3[2]:.2E}")

mpe:0.209, mad:283, mse:1.10E+05
